# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Table of Contents 💼 🗝️</p>


<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>
    
* **[1.The purpose of the project 📜](#1)**

  - A simple definition of the idea of the project
    
    
* **[2.Get Started 🔥](#2)**  

   - Start codeing ! 
    
 
* **[3.Download Ollama Server 🦙](#3)**  

    - Download Ollama and run the server

    - Pulling LLaMA 3.2 via Ollama 
      
    
* **[4.Project Settings 🔧](#4)**  
    
   - Base Class that have the project settings 
    
    
* **[5.LLM Engine Abstraction ⚙️🤖](#5)**  
    
    - Unified interface for large language model providers
    
    - Decouples model serving from pipeline logic
    
    - Allows easy switching between LLM backends
        
    
* **[6.PDF Ingestion & Metadata Enrichment 📄🧩](#6)**  
   
    - Loads raw PDF documents  
    
    - Enriches documents with consistent and searchable metadata
 
    - Keeps ingestion isolated from chunking and retrieval stages
 

* **[7.Chunking 🧱✂️](#7)**

    - Splits documents into semantically meaningful chunks
      
    - Controls chunk size and overlap to balance context and recall
      
* **[8.Hybrid Retrieval System 🔍🎯](#8)**

     - Dual retrieval strategy : Combines semantic (dense) and keyword (sparse) search
   
     - Advanced reranking: Uses cross-encoder to boost precision and select the most relevant documents from combined results


* **[9.Conversational Memory System 💬🧠](#9)**

    - Session-aware memory

    - Rolling window


* **[10.Query Rewriting & RAG Pipeline 🔄📝](#10)**

    - **Intelligent query rewriting** : Converts follow-up questions into standalone queries by replacing pronouns with actual subjects

    - **Multi-stage pipeline** : Query rewriting → Retrieval → Context formatting → Answer generation
 
* **[11.Dataset Building & Query Logging 📊💾](#11)**

    - Automatic dataset creation
  
    - Real-time logging

* **[12.RAG Evaluation System 🧪📈](#12)**

    - LLM-as-Judge evaluation
      
    - Dual evaluation metrics

<a id="1"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">The purpose of the project 📜</p>

<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>

🔘 **Definition of the problem :** Large language models (LLMs) are powerful, but they suffer from a critical limitation:
they generate answers based on their general training data, which can lead to hallucinations, outdated information, or answers that are not grounded in a specific knowledge source.

Imagine a user asking questions about a private document, a technical PDF, or internal company data.
Without a proper retrieval mechanism, the model may provide confident but incorrect answers, forcing users to manually search through documents, wasting time and increasing the risk of misinformation.

This problem becomes even more challenging in multi-turn conversations, where follow-up questions depend on previous context, making it difficult for traditional systems to understand what the user is referring to.

🔘 **The solution:** To address this challenge, this project implements a production-grade Retrieval-Augmented Generation (RAG) system that combines document retrieval, conversational memory, and large language models.

The system uses a hybrid retrieval strategy (dense + sparse retrievers) to accurately fetch relevant information from PDF documents, ensuring high recall and precision.
A query rewriting mechanism is applied to transform follow-up questions into standalone queries, enabling the retriever to correctly understand user intent even in conversational settings.

Additionally, a window-based conversational memory is integrated to maintain context across interactions without polluting the retrieval process.
The final response is generated using an open-source LLM (LLaMA 3.2 via Ollama), and the entire pipeline is evaluated using LangSmith observability and evaluation tools, focusing on answer relevance and context groundedness.

This approach results in a reliable, explainable, and enterprise-ready RAG system that delivers accurate answers strictly grounded in the provided documents.

<a id="2"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Get Started 🔥</p>

In [1]:
import sys
import torch
import platform

print("Python version:", sys.version)
print("Platform:", platform.platform())

print("\nGPU CHECK")
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)


Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35

GPU CHECK
CUDA available: True
GPU name: Tesla P100-PCIE-16GB
CUDA version: 12.6


In [2]:
pip install -q --upgrade langchain_core langchain_community langchain faiss-cpu rank_bm25 langsmith

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 9.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 93.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 96.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.0/342.0 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.6/212.6 kB 15.5 MB/s eta 0:00:00
ERROR: pip's depend

In [3]:
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
api_secret_value = user_secrets.get_secret("LANGSMITH_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = api_secret_value
os.environ["LANGCHAIN_PROJECT"] = "AI RAG System"

In [4]:
import langchain
import langchain_core
import langsmith

print("LangChain version:", langchain.__version__)
print("LangChain Core version:", langchain_core.__version__)
print("LangSmith  version:", langsmith.__version__)

LangChain version: 1.2.6
LangChain Core version: 1.2.7
LangSmith  version: 0.6.4


In [5]:
class DeviceManager:
    @staticmethod
    def get_device() -> str:
        return "GPU" if torch.cuda.is_available() else "CPU"

DEVICE = DeviceManager.get_device()
print("Device name is :",DEVICE)

Device name is : GPU


<a id="3"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Download Ollama Server 🦙</p>

In [6]:
import subprocess
import time
import shutil

class OllamaRuntime:
    @staticmethod
    def is_installed() -> bool:
        return shutil.which("ollama") is not None
    
    @staticmethod
    def install_dependencies():
        """Install required dependencies for Ollama"""
        print("Installing dependencies (zstd)...")
        try:
            subprocess.run(
                ["apt-get", "update"],
                check=True,
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
            subprocess.run(
                ["apt-get", "install", "-y", "zstd"],
                check=True
            )
        except (subprocess.CalledProcessError, FileNotFoundError):
            try:
                subprocess.run(
                    ["dnf", "install", "-y", "zstd"],
                    check=True
                )
            except (subprocess.CalledProcessError, FileNotFoundError):
                try:
                    subprocess.run(
                        ["pacman", "-S", "--noconfirm", "zstd"],
                        check=True
                    )
                except (subprocess.CalledProcessError, FileNotFoundError):
                    print("ERROR: Could not install zstd automatically.")
                    print("Please install manually:")
                    print("  - Debian/Ubuntu: sudo apt-get install zstd")
                    print("  - RHEL/CentOS/Fedora: sudo dnf install zstd")
                    print("  - Arch: sudo pacman -S zstd")
                    raise
    
    @staticmethod
    def install():
        print("Installing Ollama...")
        # Install dependencies first
        OllamaRuntime.install_dependencies()
        
        # Now install Ollama
        subprocess.run(
            "curl -fsSL https://ollama.com/install.sh | sh",
            shell=True,
            check=True
        )
    
    @staticmethod
    def start_server():
        print("Starting Ollama server...")
        subprocess.Popen(
            ["ollama", "serve"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        time.sleep(5)
    
    @staticmethod
    def pull_model(model_name: str):
        print(f"Pulling model: {model_name}")
        subprocess.run(
            ["ollama", "pull", model_name],
            check=True
        )



In [7]:
# Run installation
if not OllamaRuntime.is_installed():
    OllamaRuntime.install()

OllamaRuntime.start_server()
OllamaRuntime.pull_model("llama3.2")

Installing Ollama...
Installing dependencies (zstd)...
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 138 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 2s (351 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 129073 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...


>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%                                                         2.7%                                    38.5%######################################################                 80.3%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Starting Ollama server...
Pulling model: llama3.2


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling dde5aa3fc5ff:   2% ▕                  ▏  33 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   4% ▕                  ▏  74 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   8% ▕█                 ▏ 168 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  13% ▕██                ▏ 268 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  16% ▕██                ▏ 317 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  21% ▕███               ▏ 418 MB/2.0 GB                  

<a id="4"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Project Settings 🔧</p>

<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>
    
defines the central configuration of the project in a clean and structured way.
All core settings—such as the LLM, chunking strategy, embeddings, retriever behavior, memory window, and observability—are grouped into immutable data classes to ensure consistency and prevent accidental changes during execution.

By separating configuration from logic, the project becomes easier to maintain, debug, and scale.
Any modification to the system behavior (for example, switching models, tuning retrieval weights, or adjusting chunk sizes) can be done from a single place without touching the core pipeline code.

This approach reflects a production-oriented design, similar to how configurations are managed in real-world enterprise AI systems

In [10]:
from dataclasses import dataclass

@dataclass(frozen=True)
class BaseSettings():
    """
    Immutable base settings class.
    Any change requires explicit override.
    """

    PROJECT_NAME: str  ="Enterprise_Hybrid_RAG"
    VERSION: str = "1.0.0"


@dataclass(frozen=True)
class LLMSettings:
    PROVIDER: str = "ollama"
    MODEL_NAME: str = "llama3.2"
    BASE_URL: str = "http://localhost:11434"

    MAX_NEW_TOKENS: int = 512
    TEMPERATURE: float = 0


@dataclass(frozen=True)
class ChunkingSettings:
    CHUNK_SIZE: int = 1500
    CHUNK_OVERLAP: int = 200
    SEPARATORS: tuple = (
        "\n\n",
        "\n",
        ". ",
        " ",
    )


@dataclass(frozen=True)
class EmbeddingSettings:
    MODEL_NAME: str = "intfloat/e5-large-v2"
    NORMALIZE: bool = True
    DEVICE: str = DEVICE


@dataclass(frozen=True)
class RetrieverSettings:
    DENSE_TOP_K: int = 5
    SPARSE_TOP_K: int = 5

    DENSE_WEIGHT: float = 0.6
    SPARSE_WEIGHT: float = 0.4

    FINAL_TOP_K: int = 5


@dataclass(frozen=True)
class MemorySettings:
    WINDOW_SIZE: int = 4


@dataclass(frozen=True)
class ObservabilitySettings:
    ENABLE_LANGSMITH: bool = True
    PROJECT_NAME: str = "enterprise-hybrid-rag"




class Settings:
    base = BaseSettings()
    llm = LLMSettings()
    chunking = ChunkingSettings()
    embeddings = EmbeddingSettings()
    retriever = RetrieverSettings()
    memory = MemorySettings()
    observability = ObservabilitySettings()


Settings.base.PROJECT_NAME

'Enterprise_Hybrid_RAG'

In [11]:
pip install -q langchain_ollama

Note: you may need to restart the kernel to use updated packages.


<a id="5"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">LLM Engine Abstraction ⚙️🤖</p>


<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>
    
introduces an abstraction layer for the LLM engine, separating model loading and configuration from the core RAG pipeline.
By defining a common interface, the system can support different LLM providers without changing the pipeline logic, making the architecture more flexible and maintainable.

This design enables easy model replacement or extension (for example, switching from Ollama to another provider) while keeping the rest of the system unchanged, reflecting a clean and production-oriented engineering approach.

In [12]:
from langchain_core.language_models import BaseChatModel
from langchain_ollama import ChatOllama
from langchain_core.runnables  import Runnable

#LLM Engine Interface To decouple model serving from pipeline orchestration
class LLMEngine:
    """
    Abstract LLM Engine.
    Any LLM provider must implement this interface.
    """
    def get_model(self) -> BaseChatModel:
        raise NotImplemented



class OllamaLLMEngine(LLMEngine):
    def __init__(self , settings):
        self.settings = settings
        self._model = self._load_model()
        
    def _load_model(self) -> BaseChatModel:
        return ChatOllama(model = self.settings.MODEL_NAME , temperature = self.settings.TEMPERATURE)

    def get_model(self) ->BaseChatModel:
        return self._model
        

In [13]:
llm_engine = OllamaLLMEngine(Settings.llm)
LLM = llm_engine.get_model()
print("LLM loaded:", type(LLM))

LLM loaded: <class 'langchain_ollama.chat_models.ChatOllama'>


<a id="6"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">PDF Ingestion & Metadata Enrichment 📄🧩</p>

<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>

responsible for ingesting PDF files and converting them into raw document objects while attaching essential metadata such as source, page number, and document type.
By keeping ingestion logic separate from chunking, embeddings, and retrieval, the pipeline remains modular and easier to reason about.

Cleaning and enriching the text at this early stage ensures that downstream components operate on standardized and well-structured data, which improves traceability and debugging in later stages of the RAG system.


In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from typing import List
from langsmith import traceable , Client

2026-01-20 21:45:55.145549: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768945555.342365      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768945555.411121      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768945555.898868      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768945555.898921      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768945555.898923      55 computation_placer.cc:177] computation placer alr

In [15]:
class PDFIngestor:
    """
    Responsible ONLY for loading raw documents and attaching metadata.
    No chunking, no embeddings, no retrieval logic.
    """
    def __init__(self , pdf_path : str):
        self.pdf_path = pdf_path

    def load(self) -> List[Document]:
        loader = PyPDFLoader(self.pdf_path)
        documents = loader.load()

        return self._enrich_metadata(documents)
        
    @traceable(name="enrich_metadata" , tags=["metadata","enrich_metadata"] ,)
    def _enrich_metadata(self , documents : List[Document]) -> List[Document]:
          
        enriched_docs = []
        for doc in documents:
            
            metadata = doc.metadata or {}

            metadata.update({
                "source" : self.pdf_path,
                "page" : metadata.get("page",None),
                "document_type": "pdf"
            })
            
            enriched_docs.append(
                Document(
                    page_content= self._clean_text( doc.page_content),
                    metadata = metadata
                )
            )

        return enriched_docs
        
    @staticmethod
    def _clean_text(text : str) ->str:
        return (text
               .replace("\xa0", " ")
               .replace("\n", " ")
               .strip() 
               )
        

In [16]:
PDF_PATH = "/kaggle/input/ai-information-pdf/RAG_pdf.pdf"

pdf_ingestor = PDFIngestor(PDF_PATH)
raw_documents = pdf_ingestor.load()

In [17]:
print(f"Loaded {len(raw_documents)} pages")
print(raw_documents[1].metadata)
print(raw_documents[0].page_content[:300])

Loaded 44 pages
{'producer': 'Skia/PDF m85', 'creator': 'Chromium', 'creationdate': '2021-11-28T17:20:27+00:00', 'moddate': '2021-11-28T17:20:27+00:00', 'source': '/kaggle/input/ai-information-pdf/RAG_pdf.pdf', 'total_pages': 44, 'page': 1, 'page_label': '2', 'document_type': 'pdf'}
Artificial Intelligence Interview Questions To view the live version of the page, click here. © Copyright by Interviewbit


<a id="7"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500"> Chunking  🧱✂️</p>

<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>

handles the semantic chunking of documents, breaking long texts into manageable and meaningful chunks that can be effectively indexed and retrieved later.
By tuning chunk size, overlap, and separators, the system preserves important context while avoiding overly large inputs for the language model.

Additional metadata such as chunk IDs, indices, and text length is attached to each chunk to improve traceability, enable detailed analysis, and support observability throughout the RAG pipeline.

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import uuid

class ChunkingSettings:
    CHUNK_SIZE: int = 800
    CHUNK_OVERLAP: int = 150
    SEPARATORS = [
        "\n\n",
        "\n",
        ". ",
        " ",
        ""
    ]

class SemanticChunker:
    """
    Responsible ONLY for splitting documents into semantic chunks.
    """

    def __init__(self , settings : ChunkingSettings):
        self.settings = settings
        self.splitter = RecursiveCharacterTextSplitter(
            chunk_size = settings.CHUNK_SIZE,
            chunk_overlap = settings.CHUNK_OVERLAP,
            separators = settings.SEPARATORS
        )
    @traceable(name="make chunks" ,tags=["chunking"])
    def chunk(self , documents : List[Document] ) -> List[Document]: 
        chunks = self.splitter.split_documents(documents)
        return self._attach_chunk_metadata(chunks)
        
    @traceable(name="ttach chunk metadata" ,tags=["chunking_metadata"])
    def _attach_chunk_metadata(self , chunks : List[Document]) -> List[Document]:
        enriched_chunks = []

        for idx , chunk in enumerate (chunks):
            metadata = chunk.metadata.copy()

            metadata.update({
                "chunk_id":str(uuid.uuid4()),
                "chunk_index" : idx,
                "text_length" : len(chunk.page_content)
            })
            
            enriched_chunks.append(
                Document(
                    page_content = chunk.page_content,
                    metadata = metadata
                )
            )

        return enriched_chunks
            

In [19]:
chunker = SemanticChunker(ChunkingSettings)
chunked_documents = chunker.chunk(raw_documents)

In [20]:
print(f"Total chunks: {len(chunked_documents)}")
print("*"*20)
print(chunked_documents[0].metadata)
print("*"*20)
print(chunked_documents[0].page_content[:300])


Total chunks: 93
********************
{'producer': 'Skia/PDF m85', 'creator': 'Chromium', 'creationdate': '2021-11-28T17:20:27+00:00', 'moddate': '2021-11-28T17:20:27+00:00', 'source': '/kaggle/input/ai-information-pdf/RAG_pdf.pdf', 'total_pages': 44, 'page': 0, 'page_label': '1', 'document_type': 'pdf', 'chunk_id': '10da4c97-084f-43ed-a17a-37af231ac647', 'chunk_index': 0, 'text_length': 121}
********************
Artificial Intelligence Interview Questions To view the live version of the page, click here. © Copyright by Interviewbit


<a id="8"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500"> Hybrid Retrieval System 🔍🎯</p>

<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>

Implements a production-grade hybrid retrieval system that maximizes both recall and precision. The system operates in three stages:

**Dense Retrieval (FAISS):** Uses BGE-large-en-v1.5 embeddings (MTEB score 64+) to capture semantic similarity, retrieving documents that are conceptually related to the query even without exact keyword matches.

**Sparse Retrieval (BM25):** Employs traditional keyword-based search to ensure documents with exact term matches are captured, particularly effective for technical terms, names, and specific phrases.

**Cross-Encoder Reranking:** Applies BAAI/bge-reranker-large to re-score the combined candidate pool using query-document pair analysis. This dramatically improves precision by ranking documents based on true relevance rather than just similarity scores.

The hybrid approach ensures high recall (finding all relevant documents through dual retrieval) while the reranker ensures high precision (selecting only the most relevant from the combined pool). Normalized embeddings and GPU acceleration optimize both speed and quality. Each component is instrumented with LangSmith tracing for complete observability of the retrieval pipeline.

In [21]:
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain_core.retrievers import BaseRetriever
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
from typing import List

In [22]:
class EmbeddingEngine:
    """
    Upgraded to use BGE-large-en-v1.5 - one of the best open-source embedders.
    MTEB Score: 64+ (vs 56 for all-MiniLM-L6-v2)
    """
    def __init__(self, model_name: str = "BAAI/bge-large-en-v1.5"):
        print(f"🔄 Loading embedding model: {model_name}")
        self.model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={"device": "cuda"},
            encode_kwargs={"normalize_embeddings": True}
        )
        print("✅ Embedding model loaded!")
    
    def get(self):
        return self.model

class DenseRetriever:
    """
    Dense retrieval using FAISS.
    Now retrieves more candidates (k=50) for better recall before reranking.
    """
    def __init__(self, documents: List[Document], embeddings, k: int = 50):
        print(f"🔄 Building FAISS index for {len(documents)} documents...")
        self.vectorstore = FAISS.from_documents(documents, embeddings)
        self.retriever = self.vectorstore.as_retriever(search_kwargs={"k": k})
        print("✅ FAISS index built!")

    @traceable(name="dense retriever" , tags=["dense" , "fass"])
    def retrieve(self, query: str) -> List[Document]:
        return self.retriever.invoke(query)

class SparseRetriever:
    """
    BM25 sparse retrieval.
    Now retrieves more candidates (k=50) for better recall before reranking.
    """
    def __init__(self, documents: List[Document], k: int = 50):
        print(f"🔄 Building BM25 index for {len(documents)} documents...")
        self.retriever = BM25Retriever.from_documents(documents)
        self.retriever.k = k
        print("✅ BM25 index built!")

    @traceable(name="sparse retriever" , tages=["sparse" , "bm25"])
    def retrieve(self, query: str) -> List[Document]:
        return self.retriever.invoke(query)



class Reranker:
    """
    Cross-encoder reranker for precision boost.
    This is what separates good RAG from great RAG.
    """    
    def __init__(self, model_name :str = "BAAI/bge-reranker-large"):
        print(f"🔄 Loading reranker model: {model_name}")
        self.model = CrossEncoder(model_name , device="cuda")
        print("✅ Reranker loaded!")

    @traceable(name="ranker" , tags=["ranker"])
    def rerank(self, query : str, documents: List[Document], top_k : int =5) -> List[Document]:
        
        """
        Rerank documents using cross-encoder.
        Returns top_k most relevant documents with scores.
        """
        if not documents:
            return []

        pairs = [[query , doc.page_content]   for doc in documents]
        scores = self.model.predict(pairs)
        doc_score_pairs = list(zip(documents, scores))
        doc_score_pairs.sort(key= lambda x : x[1] , reverse=True)

        reranked_docs = []
        for doc , score in doc_score_pairs[:top_k]:
            new_metadata  = doc.metadata.copy()
            new_metadata['reranker_score'] = float(score)
            
            reranked_doc =Document(
                page_content= doc.page_content,
                metadata = new_metadata
            )
            reranked_docs.append(reranked_doc)
        
        return reranked_docs       



In [23]:
class HybridRetriever(BaseRetriever):
    """
    Production-grade hybrid retriever:
    1. Dense retrieval (50 candidates)
    2. Sparse retrieval (50 candidates)
    3. RRF fusion (merge to top 20)
    4. Cross-encoder reranking (final top K)
    """
    dense: DenseRetriever
    sparse: SparseRetriever
    reranker: Reranker
    k: int = 5  # Final number of documents to return
    fusion_k: int = 20  # Number of docs to pass to reranker
    c: int = 60  # RRF constant
    
    def __init__(
        self,
        dense_retriever: DenseRetriever,
        sparse_retriever: SparseRetriever,
        reranker: Reranker,
        k: int = 5,
        fusion_k: int = 20
    ):
        super().__init__(
            dense=dense_retriever,
            sparse=sparse_retriever,
            reranker=reranker,
            k=k,
            fusion_k=fusion_k
        )
        self.dense = dense_retriever
        self.sparse = sparse_retriever
        self.reranker = reranker
        self.k = k
        self.fusion_k = fusion_k

    @traceable(name="hybrid_retrieval_pipeline" , tags=["retrieval", "hybrid"] , metadata={"component": "retriever"})
    def _get_relevant_documents(self, query: str) -> List[Document]:
        """
        Enhanced retrieval pipeline with reranking.
        """
        print(f"\n{'='*80}")
        print(f"🔍 Query: '{query}'")
        print(f"{'='*80}")
        
        # Stage 1: Retrieve candidates from both retrievers
        print("📊 Stage 1: Retrieving candidates...")
        dense_docs = self.dense.retrieve(query)
        sparse_docs = self.sparse.retrieve(query)
        print(f"   Dense: {len(dense_docs)} docs | Sparse: {len(sparse_docs)} docs")
        # Stage 2: RRF Fusion
        print("🔀 Stage 2: Reciprocal Rank Fusion...")
        doc_scores = {}
        # Score dense results
        for rank , doc in enumerate(dense_docs):
            doc_id = doc.metadata['chunk_id']
            doc_scores[doc_id]=doc_scores.get(doc_id,0) + 1 / (rank + self.c)
        
        # Score sparse results
        for rank, doc in enumerate(sparse_docs):
            doc_id = doc.metadata["chunk_id"]
            if doc_id not in doc_scores:
                doc_scores[doc_id]= 1 / (rank + self.c)
            else:
                doc_scores[doc_id]+= 1 / (rank + self.c)
                
        all_docs = {doc.metadata["chunk_id"]:doc for doc in dense_docs + sparse_docs}                 
        sorted_ids = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
        print(len(all_docs))
        fusion_docs = [all_docs[doc_id]  for doc_id , score in sorted_ids[:self.fusion_k]]
        print(f"   Fused to top {len(fusion_docs)} candidates")

        # Stage 3: Reranking
        print("🎯 Stage 3: Cross-encoder reranking...")
        final_docs = self.reranker.rerank(query,fusion_docs,top_k=self.k)
        print(f"   ✅ Final top {len(final_docs)} documents selected")
        print(f"{'='*80}\n")

        return final_docs

In [24]:
# Step 1: Initialize embedding engine (UPGRADED MODEL)
embedding_engine = EmbeddingEngine(model_name="BAAI/bge-large-en-v1.5")
embeddings = embedding_engine.get()
    
# Step 2: Initialize retrievers
dense = DenseRetriever(chunked_documents, embeddings, k=50)
sparse = SparseRetriever(chunked_documents, k=50)
    
# Step 3: Initialize reranker (NEW!)
reranker = Reranker(model_name="BAAI/bge-reranker-large")

🔄 Loading embedding model: BAAI/bge-large-en-v1.5


/tmp/ipykernel_55/1246013275.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  self.model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✅ Reranker loaded!


In [25]:
    # Step 4: Create hybrid retriever
hybrid_retriever = HybridRetriever(
    dense_retriever=dense,
    sparse_retriever=sparse,
    reranker=reranker,
    k=5,  # Return top 5 final documents
    fusion_k=20  # Pass top 20 to reranker
)

In [26]:
query = "What is a 'Bayesian Network' and what is it used for?"
results = hybrid_retriever._get_relevant_documents(query)


🔍 Query: 'What is a 'Bayesian Network' and what is it used for?'
📊 Stage 1: Retrieving candidates...
   Dense: 50 docs | Sparse: 50 docs
🔀 Stage 2: Reciprocal Rank Fusion...
66
   Fused to top 20 candidates
🎯 Stage 3: Cross-encoder reranking...
   ✅ Final top 5 documents selected



In [27]:
print("\n" + "="*80)
print("📄 TOP RESULTS:")
print("="*80)
for i, doc in enumerate(results, 1):
    print(f"\n[{i}] Reranker Score: {doc.metadata.get('reranker_score', 0):.4f}")
    print(f"Page: {doc.metadata.get('page', 'N/A')}")
    print(f"Content Preview: {doc.page_content}...")
    print("-" * 80)


📄 TOP RESULTS:

[1] Reranker Score: 0.9988
Page: 20
Content Preview: Artificial Intelligence Interview Questions 16.   What are Bayesian networks? A Bayesian network is a probabilistic graphical model based on a set of variables and their dependencies, represented in the form of an acyclic graph. Bayesian networks are based on probability distribution, and they predict outcomes and detect anomalies using probability theory. The Bayesian networks are used to perform tasks such as prediction, detecting anomalies, reasoning, gaining insights, diagnostics, and decision-making. A Bayesian network, for example, could be used to illustrate the probability correlations between diseases and symptoms. The network may be used to calculate the chances of certain diseases being present based on symptoms. 17...
--------------------------------------------------------------------------------

[2] Reranker Score: 0.4066
Page: 20
Content Preview: . The network may be used to calculate the chances of c

<a id="8"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Conversational Memory System 💬🧠</p>


<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>

Provides stateful conversation management for the RAG system, enabling multi-turn dialogues where the system can understand follow-up questions and maintain context across interactions.
The MemoryStore implements a sliding window approach that keeps the last 4 exchanges **(8 messages: 4 user + 4 assistant)** per session. This window size balances context retention with token efficiency—providing enough history for coherent follow-ups while preventing excessive context length that could slow inference or exceed model limits.
Each session is isolated by a unique session_id, allowing the system to handle multiple concurrent users without cross-contamination of conversation histories. When the memory exceeds the window size, older messages are automatically pruned, ensuring the system always operates within optimal context boundaries.
The with_memory wrapper integrates any LangChain chain with this memory system, automatically injecting conversation history into prompts and storing new exchanges. The clear_memory utility enables manual session resets when needed (e.g., topic changes, debugging, or user logout).
This memory architecture is fully traceable through LangSmith, allowing you to inspect how conversation history influences model responses and debug issues related to context carryover or pronoun resolution in follow-up questions.

In [28]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import BaseMessage


class MemoryStore:
    """
    Session-aware memory store.
    Each session has its own rolling window memory.
    """
    def __init__(self, window_size : int=4):
        self.window_size = window_size
        self.store = {}

    def get_session_history(self, session_id : str) -> InMemoryChatMessageHistory:
        if session_id not in self.store:
            self.store[session_id] = InMemoryChatMessageHistory()

        history = self.store[session_id]

        if len(history.messages) > self.window_size*2:
            history.messages = history.messages[-self.window_size*2:]

        return history


memory_store = MemoryStore(window_size=4)

@traceable(name="chain with memory" , tags=["memory"])
def with_memory(chain):
    return RunnableWithMessageHistory(
        chain,
        memory_store.get_session_history,
        input_messages_key="question",
        history_messages_key="chat_history",
    )


def clear_memory(session_id: str):
    """Clear memory for a specific session."""
    if session_id in memory_store.store:
        memory_store.store[session_id].clear()
        print(f"✅ Memory cleared for session: {session_id}")
    else:
        print(f"❌ No memory found for session: {session_id}")


<a id="10"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Query Rewriting & RAG Pipeline 🔄📝</p>

<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>

Implements a sophisticated conversational RAG pipeline that handles multi-turn dialogues seamlessly. The system addresses a critical challenge in conversational AI: **follow-up questions often contain pronouns ("it", "this", "they") that lack meaning without conversation context.**

**Query Rewriting Stage :** Uses **Qwen 2.5 (7B)** model with strict rules to transform context-dependent questions into standalone queries. For example, after discussing "artificial intelligence," the follow-up "What is its importance?" becomes "What is artificial intelligence's importance?" This rewriting is conservative—it only modifies questions containing pronouns, preserving the original structure and intent. The rewriter accesses chat history to identify referents and operates with temperature=0 for consistent, deterministic output.
RAG Pipeline Architecture: The system operates as a four-step sequential chain:

Query Rewriting: Transforms the **user's question** into a **standalone version using conversation history**

**Hybrid Retrieval :** Invokes the hybrid retriever with the rewritten query to find relevant document chunks

**Context Formatting :** Formats retrieved documents with page numbers for citation and traceability

**Answer Generation :** Feeds the formatted context and original question to the LLM **(Llama 3.2)**, which generates a grounded answer strictly based on the provided context


In [29]:
print("🔄 Pulling Qwen 2.5 (7B) model...")
OllamaRuntime.pull_model("qwen2.5:7b")
print("✅ Qwen 2.5 model ready!")

🔄 Pulling Qwen 2.5 (7B) model...
Pulling model: qwen2.5:7b


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 2bada8a74506:   1% ▕                  ▏  37 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   2% ▕                  ▏  73 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   4% ▕                  ▏ 169 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   6% ▕█                 ▏ 270 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   7% ▕█                 ▏ 318 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   9% ▕█                 ▏ 421 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:  11% ▕█                 ▏ 517 MB/4.7 GB                  pulling manifest 
pulling 2bada8a745

✅ Qwen 2.5 model ready!


pulling manifest 
pulling 2bada8a74506: 100% ▕██████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b: 100% ▕██████████████████▏   68 B                         
pulling eb4402837c78: 100% ▕██████████████████▏ 1.5 KB                         
pulling 832dd9e00a68: 100% ▕██████████████████▏  11 KB                         
pulling 2f15b3218f05: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest ⠼ pulling manifest 
pulling 2bada8a74506: 100% ▕██████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b: 100% ▕██████████████████▏   68 B                         
pulling eb4402837c78: 100% ▕██████████████████▏ 1.5 KB                         
pulling 832dd9e00a68: 100% ▕██████████████████▏  11 KB                         
pulling 2f15b3218f05: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [30]:
rewriter_llm = ChatOllama(
    model="qwen2.5:7b",
    temperature=0,
    num_ctx=2048  # Context window - enough for chat history
)

print("✅ Rewriter LLM loaded with Qwen 2.5 (7B)")


✅ Rewriter LLM loaded with Qwen 2.5 (7B)


In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

@traceable(name="context formatting" , tags = ["context"])
def format_context(docs):
    return "\n\n".join(f"[Page {doc.metadata.get("page")}] \n {doc.page_content}" for doc in docs)


from langchain_core.prompts import MessagesPlaceholder

rag_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are a senior AI assistant using Retrieval-Augmented Generation (RAG).
Use the conversation history to understand follow-up questions.
Answer ONLY using the provided context.
If the answer is not in the context, say you don't know.
Always mention the page from which the information was retrieved."""
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    (
        "human",
        """Context:
           {context}

           Question:
           {question}"""
    )
])


query_rewriter_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are a query reformulation assistant. Your job is to rewrite follow-up questions to be standalone.

CRITICAL RULES:
1. If the question has NO pronouns (it, its, this, that, they, their, them), return it EXACTLY as written
2. If it HAS pronouns, replace them with the actual subject from the chat history
3. NEVER remove words like "What is", "Does", "How", etc.
4. NEVER shorten or modify the question structure
5. Look at the MOST RECENT user message to determine what pronouns refer to

Examples:
Input: "What is artificial intelligence"
Output: "What is artificial intelligence"

Input: "What is its importance?" (after AI discussion)
Output: "What is artificial intelligence's importance?"

Input: "Does it work?" (after ML discussion)
Output: "Does machine learning work?"

Remember: ONLY replace pronouns. Keep everything else identical.
"""
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])


query_rewriter_chain = query_rewriter_prompt | rewriter_llm | StrOutputParser()
query_rewriter_with_memory = with_memory(query_rewriter_chain)

print("✅ Query Rewriter Chain created (Qwen 2.5)")


@traceable(name="retrieve and format" , tags = ["retriever","context foramtting" , 'hybrid retriever'])
def retrieve_and_format(x):
    docs = hybrid_retriever.invoke(x["standalone_question"])
    return format_context(docs)



rag_chain = (
    RunnablePassthrough.assign(
        standalone_question=query_rewriter_with_memory  # Step 1: Rewrite query
    )
    | RunnablePassthrough.assign(
        context=RunnableLambda(retrieve_and_format)  # Step 2: Retrieve with rewritten query
    )
    | rag_prompt  # Step 3: Build prompt
    | LLM  # Step 4: Generate answer
    | StrOutputParser()
)

✅ Query Rewriter Chain created (Qwen 2.5)


In [32]:
rag_chain_with_memory = with_memory(rag_chain)

In [33]:
session_id = "finally"

response = rag_chain_with_memory.invoke(
    {"question": "What is the AI?"},
    config={"configurable": {"session_id": session_id}}
)

print("ANSWER \n", response)


🔍 Query: 'What is the artificial intelligence?'
📊 Stage 1: Retrieving candidates...
   Dense: 50 docs | Sparse: 50 docs
🔀 Stage 2: Reciprocal Rank Fusion...
67
   Fused to top 20 candidates
🎯 Stage 3: Cross-encoder reranking...
   ✅ Final top 5 documents selected

ANSWER 
 According to Page 4, Artificial Intelligence (AI) is a field of data science that tries to impart human-like intelligence and wisdom to machines. It makes use of subfields like Machine learning and Deep learning to achieve the purpose.


<a id="11"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Dataset Building & Query Logging 📊💾</p>

<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>

Implements production-grade data collection infrastructure that automatically builds high-quality evaluation datasets from real user interactions. This system solves a critical challenge in RAG development: obtaining representative test data that reflects actual usage patterns.

**Dataset Management :** The code first attempts to read an existing dataset named "rag-real-user-queries" from LangSmith. If the dataset doesn't exist, it creates a new one with a descriptive label. This idempotent approach ensures the system works reliably whether running for the first time or resuming data collection.

**Query Logging Function: The ask_and_log function serves dual purposes :**

**Executes RAG Query :** Invokes the full conversational RAG pipeline with memory support, processing the user's question through query rewriting, hybrid retrieval, and answer generation

**Logs to Dataset :** Automatically captures the complete interaction in LangSmith, storing both inputs (question, session_id, system configuration) and outputs (generated answer) as structured examples


This automated logging creates a continuously growing evaluation dataset that reflects real-world usage patterns, difficult questions, edge cases, and common query types. The accumulated data can be used for regression testing, quality monitoring, model fine-tuning, and benchmarking system improvements—all without manual curation effort.

In [35]:
from langsmith import Client

client = Client()
DATASET_NAME = "rag-real-user-queries"

try:
    dataset = client.read_dataset(dataset_name=DATASET_NAME)
    print("📂 Using existing dataset")

except:
    dataset = client.create_dataset(dataset_name=DATASET_NAME,
                                    description="Real user questions and LLM answers from RAG system")
    print("✅ Dataset created")

✅ Dataset created


In [36]:
def ask_and_log(question: str, session_id: str):
    response = rag_chain_with_memory.invoke(
        {"question": question},
        config={"configurable": {"session_id": session_id}}
    )

    # Add to LangSmith Dataset
    client.create_example(
        inputs={
            "question": question,
            "session_id": session_id,
            "retriever": "hybrid",
            "model": "llama3.2",
        },
        outputs={
            "answer": response
        },
        dataset_id=dataset.id
    )

    return response


In [37]:
response = ask_and_log(
    question="What is the AI?",
    session_id=session_id
)

response


🔍 Query: 'What is artificial intelligence?'
📊 Stage 1: Retrieving candidates...
   Dense: 50 docs | Sparse: 50 docs
🔀 Stage 2: Reciprocal Rank Fusion...
65
   Fused to top 20 candidates
🎯 Stage 3: Cross-encoder reranking...
   ✅ Final top 5 documents selected



'According to Page 1, Artificial Intelligence (AI) is defined as "systems that act like humans".'

<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>
    
<p style="font-size: 18px;">
  <span style="color: red; font-size: 30px; font-weight: bold;">note :</span> 
   See how the question will be rewritten in the next cells.
</p>

In [38]:
response = ask_and_log(
    question="Does it have anything to do with machine learning?",
    session_id=session_id
)

response


🔍 Query: 'Does artificial intelligence have anything to do with machine learning?'
📊 Stage 1: Retrieving candidates...
   Dense: 50 docs | Sparse: 50 docs
🔀 Stage 2: Reciprocal Rank Fusion...
69
   Fused to top 20 candidates
🎯 Stage 3: Cross-encoder reranking...
   ✅ Final top 5 documents selected



'According to Page 11, Artificial Intelligence (AI) and Machine Learning are related in that Machine Learning is a subset of Artificial Intelligence. In other words, AI incorporates Machine Learning.'

In [39]:
response = ask_and_log(
    question="Amazing! What does it have to do with medicine?",
    session_id=session_id
)

response


🔍 Query: 'What does artificial intelligence have to do with medicine?'
📊 Stage 1: Retrieving candidates...
   Dense: 50 docs | Sparse: 50 docs
🔀 Stage 2: Reciprocal Rank Fusion...
71
   Fused to top 20 candidates
🎯 Stage 3: Cross-encoder reranking...
   ✅ Final top 5 documents selected



'According to Page 6, Artificial Intelligence (AI) has come out as a reliable friend of doctors. From intelligent testing to medical recommendations, they assist medical professionals in every possible way.'

In [40]:

response = ask_and_log(
    question="Are You Know Mohamed Wasef?",
    session_id=session_id
)

response


🔍 Query: 'Are you know Mohamed Wasef?'
📊 Stage 1: Retrieving candidates...
   Dense: 50 docs | Sparse: 50 docs
🔀 Stage 2: Reciprocal Rank Fusion...
71
   Fused to top 20 candidates
🎯 Stage 3: Cross-encoder reranking...
   ✅ Final top 5 documents selected



"I don't know who Mohamed Wasef is."

<a id="12"></a>
# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">RAG Evaluation System 🧪📈</p>

<div style = 'border : 3px solid #E8D2B0; background-color:#F5E6D0;padding:10px; border-radius: 25px;'>

Implements automated quality assessment for the RAG system using LLM-based evaluators that provide objective, consistent scoring of system outputs. This evaluation framework addresses two critical quality dimensions:

**Answer Relevance Evaluator :** Assesses whether the generated answer actually addresses the user's question. The evaluator uses Qwen 2.5 configured for JSON output (temperature=0 for consistency) to score answers on a 0-1 scale. A score of 1.0 indicates the answer is perfectly relevant and directly addresses the question, while lower scores indicate partial relevance, tangential information, or off-topic responses. Each score includes reasoning that explains the assessment, enabling diagnostic analysis of failure modes.

**Groundedness Evaluator:** Performs strict hallucination detection by verifying that every claim in the answer is supported by the retrieved context. This binary evaluator (0 or 1) ensures the RAG system adheres to its core principle: answer only from provided documents. The evaluator checks multiple locations for context (run.inputs, run.outputs) to handle different pipeline configurations. A score of 1 means the answer is fully grounded; 0 indicates unsupported claims or fabricated information.

In [43]:
from langsmith import evaluate
from langchain_core.output_parsers import JsonOutputParser
#relevance prompt
relevance_prompt = ChatPromptTemplate.from_template("""
    You are an evaluator.
    Question:
    {question}
    Answer:
    {answer}
    Evaluate whether the answer is relevant to the question.
    Return a JSON with:
    - score: number between 0 and 1
    - reasoning: short explanation
    """)

judge_llm = ChatOllama(
    model="qwen2.5:7b",
    temperature=0,
    format="json"
)

def answer_relevance_evaluator(run, example):
    chain = relevance_prompt | judge_llm | JsonOutputParser()
    result = chain.invoke({
        "question": example.inputs["question"],
        "answer": run.outputs.get("output", run.outputs.get("answer", "")) 
    })
    return {
        "key": "answer_relevance",
        "score": result["score"],
        "comment": result["reasoning"]
    }

# Context Groundedness
grounded_prompt = ChatPromptTemplate.from_template("""
    You are a strict RAG evaluator.
    Context:
    {context}
    Answer:
    {answer}
    Is the answer fully supported by the context?
    Return JSON:
    - score: 0 or 1
    - reasoning
    """)

def groundedness_evaluator(run, example):
    chain = grounded_prompt | judge_llm | JsonOutputParser()
    
    
    context = (
        run.inputs.get("context") or 
        run.outputs.get("context") or 
        "No context available"
    )
    
    result = chain.invoke({
        "context": context,
        "answer": run.outputs.get("output", run.outputs.get("answer", "")) 
    })
    return {
        "key": "groundedness",
        "score": result["score"],
        "comment": result["reasoning"]
    }



def create_target_function(rag_chain_with_memory, session_id="eval-session"):
    """
    Wrapper function that LangSmith evaluate() can call.
    
    This is REQUIRED because rag_chain_with_memory expects:
      {"question": "...", ...}
    
    But LangSmith provides:
      {"question": "..."}  # From dataset
    
    And expects back:
      {"output": "..."}  # Standard format
    """
    def predict(inputs: dict) -> dict:
        """
        Args:
            inputs: {"question": "..."}  (from dataset)
        
        Returns:
            {"output": "...", "answer": "..."}
        """
        question = inputs["question"]
        
        # Invoke your RAG chain
        response = rag_chain_with_memory.invoke(
            {"question": question},
            config={
                "configurable": {"session_id": session_id},
                "tags": ["evaluation"]
            }
        )
        
        return {
            "output": response,
            "answer": response   
        }
    
    return predict




target_function = create_target_function(
    rag_chain_with_memory=rag_chain_with_memory,
    session_id="evaluation-run-1"
)

results = evaluate(
    target_function,  
    data=DATASET_NAME,
    evaluators=[
        answer_relevance_evaluator,
        groundedness_evaluator,
    ],
    experiment_prefix="rag-v1"
)

print("Evaluation complete!")
print(results)

View the evaluation results for experiment: 'rag-v1-ad64535d' at:
https://smith.langchain.com/o/2c920183-7a67-44fc-8243-4da43415f383/datasets/9a2c6d7d-e438-42d2-a493-43c6df3b79f4/compare?selectedSessions=74692148-98e9-4385-aee9-af3e73b6a01e




0it [00:00, ?it/s]


🔍 Query: 'Are you familiar with Mohamed Wasef?'
📊 Stage 1: Retrieving candidates...
   Dense: 50 docs | Sparse: 50 docs
🔀 Stage 2: Reciprocal Rank Fusion...
75
   Fused to top 20 candidates
🎯 Stage 3: Cross-encoder reranking...
   ✅ Final top 5 documents selected


🔍 Query: 'What does artificial intelligence have to do with medicine?'
📊 Stage 1: Retrieving candidates...
   Dense: 50 docs | Sparse: 50 docs
🔀 Stage 2: Reciprocal Rank Fusion...
71
   Fused to top 20 candidates
🎯 Stage 3: Cross-encoder reranking...
   ✅ Final top 5 documents selected


🔍 Query: 'Does artificial intelligence have anything to do with machine learning?'
📊 Stage 1: Retrieving candidates...
   Dense: 50 docs | Sparse: 50 docs
🔀 Stage 2: Reciprocal Rank Fusion...
69
   Fused to top 20 candidates
🎯 Stage 3: Cross-encoder reranking...
   ✅ Final top 5 documents selected


🔍 Query: 'What is artificial intelligence?'
📊 Stage 1: Retrieving candidates...
   Dense: 50 docs | Sparse: 50 docs
🔀 Stage 2: Reciprocal Rank 

# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Interactive Streaming Mode</p>

In [ ]:
session_id = "linkedin_demo"

print(f"--- 🚀 Welcome to: [Your RAG Wingman] 🚀 ---")
print("Type 'exit' or 'quit' to stop the chat.\n")

while True:
    user_query = input("👤 User: ")
    
    if user_query.lower() in ['exit', 'quit', 'bye']:
        print("🤖 Assistant: Good luck with your interview! Bye!")
        break

    print("🤖 Assistant: ", end="", flush=True)

    chunks = []
    for chunk in rag_chain_with_memory.stream(
        {"question": user_query},
        config={"configurable": {"session_id": session_id}}
    ):
        print(chunk, end="", flush=True)
        chunks.append(chunk)
        
    print("\n" + "-"*50)

--- 🚀 Welcome to: [Your RAG Wingman] 🚀 ---
Type 'exit' or 'quit' to stop the chat.




# <p style="padding:10px;background-color:#E8D2B0 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Thanks for watching until the end ❤️ </p>